## Step2. 合并lora权重与模型权重

**本示例是基于Step1——lora微调模型后的合并权重示例。需要拥有base模型（此处为：qwen2-7B-instruct）以及基于base模型 lora微调后的适配器权重（可以参考本notebook 挂载的Example-Data下的'checkpoint-216'）。合并两者的权重后，可以像使用base模型一样使用微调后的模型，可以简单的在Step3.推理 中更换模型路径为微调后的模型，即可使用微调后的模型进行评测。**

In [1]:
!pip install transformers>=4.37.0
!pip install peft

Looking in indexes: https://pypi.tuna.tsinghua.edu.cn/simple
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 296.4/296.4 kB 3.9 MB/s eta 0:00:00a 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 797.2/797.2 MB 1.1 MB/s eta 0:00:0000:0100:02
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 124.2/124.2 MB 6.0 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 8.3 MB/s eta 0:00:00a 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.1/14.1 MB 10.1 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 1.4 MB/s eta 0:00:0000:0100:02
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 196.0/196.0 MB 5.3 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.2/6.2 MB 15.0 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 121.6/121.6 MB 8.8 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 176.2/176.2 MB 7.2 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

### 导包

In [2]:
from transformers import AutoModelForCausalLM,AutoTokenizer,Trainer,TrainingArguments
from peft import LoraConfig,TaskType,get_peft_model,PeftModel

/opt/mamba/lib/python3.10/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


### 合并模型并保存

In [3]:
device="gpu"
#设置base模型导入路径
model_path="/bohr/model-i7fa/v1/"
#导入模型
model=AutoModelForCausalLM.from_pretrained(
    pretrained_model_name_or_path=model_path,
    torch_dtype="auto",
    device_map="auto"
)

#导入tokenizer
tokenizer=AutoTokenizer.from_pretrained(model_path)
#定义lora路径和输出路径
lora_path="/bohr/exampleData-pi6b/v6/checkpoint-216/"
output_path="./personal/qwen2-7B-Instruct-merge-lora"


lora_model=PeftModel.from_pretrained(
    model,
    lora_path
)


print("applying the LoRA")
model=lora_model.merge_and_unload()
model.save_pretrained(output_path)
tokenizer.save_pretrained(output_path)
print("done")

Loading checkpoint shards: 100%|██████████| 4/4 [00:30<00:00,  7.50s/it]
applying the LoRA
done


### 【补充】平台使用帮助
baseline中模型的输出结果、微调/合并示例中模型权重的保存位置都可以根据选手的需要进行修改，可以考虑保存到个人的文件夹下（/personal/），然后再挂载到创建的数据集中。
平台数据集的使用规则可以参考：
https://bohrium-doc.dp.tech/docs/userguide/Dataset/